In [ ]:
%run Utilities.ipynb
%run DemoLanguages.ipynb

In [ ]:
def BB84_DEMO(target_bits:int, lang:str='EN', speed:int=1, validation:float=0.0)->str:
    """
    When run, produces an animation of two parties
    communicating and obtaining same random bit-string.
    This is done via BB84 protocol.
    
    Returns obtained key as a string consisting of '0's and '1's. 
    
    target_bits:  Desired length of one-time key both parties
        obtain. Due to probabilistic nature of process,
        no guarantees are given.
    lang:   Language desired for the demo. If not supported
        or blank, defaults to English. Uses ISO 639-1 coding.
    speed:  Speed desired for the text change,
        with 1 slowest and 5 fastest (no waiting between lines.)
    validation:   If >0, that ratio of resulting bits
        will be compared over classical channel.
        Takes values between [0, 0.5], with 0.5 being half the bits.
        NOT YET IMPLEMENTED!
    """
    # Input cleaning and control
    if(validation<0): validation = 0.0
    if(validation>0.5): validation = 0.5
    target = int(target_bits*(2/(1-validation)))
    lang = lang.upper()
    if(lang not in supported_languages):
        lang = 'EN'
    if(speed<1): speed=1
    if(speed>5): speed=5
    speed = (5-speed)/2
    
    # Custom variables, helper functions
    col_length = 44 # Each actor has approximately col_length wide columns.
    c2 = ' '*col_length # Moving us to 2nd column
    def read_stop():
        print()
        sleep(speed)
    anim_actn = lambda: sleep(speed/12)
    anim_fast = lambda: sleep(speed/36)
    cent      = lambda x: x.upper().center(col_length)
    def bob_print(s:str):
        for i in s.splitlines():
            print(c2+i)
    def common_print(s:str):
        for i in s.splitlines():
            print("\t\t"+i)
    
    alice   = iter(a_lines[lang])
    bob     = iter(b_lines[lang])
    common  = iter(c_lines[lang])
    names   = g_names[lang]
    
    # Alice has random bits and directions,
    # Bob has random directions, which he makes measurements accordingly.
    alice_bits = grb(target)
    alice_directions = grb(target)
    bob_directions = grb(target)
    bob_bits = ''
    
    ### Start printing ###
    print(cent(names[0])+cent(names[1]))
    print("="*(2*col_length))
    
    print(next(alice))
    read_stop()
    
    print(next(alice).format(Bob=names[1]))
    i = 0
    while(i*32<len(alice_bits)):
        print('    '+alice_bits[32*i:min(32*(i+1),len(alice_bits))])
        i += 1
    read_stop()
    
    print(next(alice))
    aqs = polarizer(alice_directions)
    i = 0
    while(i*32<len(aqs)):
        print('    '+aqs[32*i:min(32*(i+1),len(aqs))])
        i += 1
    read_stop()
    
    ##############################
    ## Actual BB84 happens here ##
    ##############################
    q_comm = '|'
    for i in range(target):
        pair = alice_bits[i]+alice_directions[i]
        if(pair=='00'):
            q_comm += '0'
        elif(pair=='10'):
            q_comm += '1'
        elif(pair=='01'):
            q_comm += '+'
        elif(pair=='11'):
            q_comm += '-'
        bob_bits += BB84(*pair, bob_directions[i])    
    q_comm += '>'
    ##############################
    ## Actual BB84 happens here ##
    ##############################
    
    q_comm = (('-'*8)+(' '*2)+q_comm+(' '*2)+('-'*7)+'>').center(col_length*2)
    print(next(alice))
    for l in q_comm:
        print(l, end='')
        if(l=='-'): anim_actn()
        else: anim_fast()
    print()
    read_stop()
    
    ### Bob starts talking now ###
    bob_print(next(bob).format(Alice=names[0]))
    bqs = polarizer(bob_directions)
    i = 0
    while(i*32<len(bqs)):
        print(c2+'    '+bqs[32*i:min(32*(i+1),len(bqs))])
        i += 1
    read_stop()
    
    bob_print(next(bob))
    i = 0
    while(i*32<len(bob_bits)):
        print(c2+'    '+bob_bits[32*i:min(32*(i+1),len(bob_bits))])
        i += 1
    read_stop()
    
    ### Communication over unsecure classical channel ###
    common_print(next(common))
    print('-{}:'.format(names[0]).rjust(8)+'\t'+aqs)
    anim_actn()
    common_print(bqs+'\t'+':{}-'.format(names[1]).ljust(8))
    
    print('\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('=', end='')
            anim_actn()
        else:
            print(' ', end='')
    read_stop()
    
    ### Determining matching directions ###
    common_print(next(common))
    print('\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('\u2193', end='')
            anim_fast()
        else:
            print(' ', end='')
    read_stop()
    
    ### Obtaining common bits ###
    print('-{}:'.format(names[0]).rjust(8)+'\t'+alice_bits)
    common_print(bob_bits+'\t'+':{}-'.format(names[1]).ljust(8))
    print('\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('\u2193', end='')
            anim_fast()
        else:
            print(' ', end='')
    print('\n\t\t', end='')
    count = 0
    key = ''
    for i in zip(alice_directions, bob_directions, alice_bits, bob_bits):
        if(i[0]==i[1]):
            print(i[2], end='')
            anim_actn()
            count += 1
            key += i[2]
        else:
            print(' ', end='')
    
    print('\n\n')
    print(next(alice).format(actual=count, target=target_bits))
    read_stop()

    ### Concluding remarks ###
    bob_print(next(bob))
    read_stop()
    print(next(alice))
    read_stop()
    print(next(common).center(2*col_length))
    
    return key

    # Possible improvements:
    #   * comparison of selected bits can be shown
    #   * additional protocols can be added